In [30]:
import pandas as pd
from datetime import datetime
from datetime import datetime

In [40]:
df = pd.read_parquet("../data/AAPL.parquet")
df

,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2020-01-02,AAPL,71.72,72.78,71.47,72.72,135480400
2020-01-03,AAPL,71.94,72.77,71.78,72.01,146322800
2020-01-06,AAPL,71.13,72.62,70.88,72.58,118387200
2020-01-07,AAPL,72.59,72.85,72.02,72.24,108872000
2020-01-08,AAPL,71.94,73.71,71.94,73.40,132079200
...,...,...,...,...,...,...
2025-04-16,AAPL,198.36,200.70,192.37,194.27,59732400
2025-04-17,AAPL,197.20,198.83,194.42,196.98,51334300
2025-04-21,AAPL,193.27,193.80,189.81,193.16,46742500


In [41]:
df1 = df.loc["2025-04-01":, :].head(10).reset_index()
df1

,Date,Ticker,Open,High,Low,Close,Volume
0,2025-04-01,AAPL,219.81,223.68,218.90,223.19,36412700
1,2025-04-02,AAPL,221.32,225.19,221.02,223.89,35905900
2,2025-04-03,AAPL,205.54,207.49,201.25,203.19,103419000
3,2025-04-04,AAPL,193.89,199.88,187.34,188.38,125910900
4,2025-04-07,AAPL,177.20,194.15,174.62,181.46,160466300
5,2025-04-08,AAPL,186.70,190.34,169.21,172.42,120859500
6,2025-04-09,AAPL,171.95,200.61,171.89,198.85,184395900
7,2025-04-10,AAPL,189.07,194.78,183.00,190.42,121880000
8,2025-04-11,AAPL,186.10,199.54,186.06,198.15,87435900
9,2025-04-14,AAPL,211.44,212.94,201.16,202.52,101352900


In [6]:
df1.columns = [col.lower() for col in df1.columns]
df1

,date,ticker,open,high,low,close,volume
0,2025-04-01,AAPL,219.81,223.68,218.90,223.19,36412700
1,2025-04-02,AAPL,221.32,225.19,221.02,223.89,35905900
2,2025-04-03,AAPL,205.54,207.49,201.25,203.19,103419000
3,2025-04-04,AAPL,193.89,199.88,187.34,188.38,125910900
4,2025-04-07,AAPL,177.20,194.15,174.62,181.46,160466300
5,2025-04-08,AAPL,186.70,190.34,169.21,172.42,120859500
6,2025-04-09,AAPL,171.95,200.61,171.89,198.85,184395900
7,2025-04-10,AAPL,189.07,194.78,183.00,190.42,121880000
8,2025-04-11,AAPL,186.10,199.54,186.06,198.15,87435900
9,2025-04-14,AAPL,211.44,212.94,201.16,202.52,101352900


In [7]:
df1["entry_signal"] = [
    "wait",
    "buy",
    "buy",
    "wait",
    "wait",
    "buy",
    "wait",
    "wait",
    "wait",
    "buy",
]
df1["exit_signal"] = [
    "sell",
    "wait",
    "wait",
    "wait",
    "sell",
    "wait",
    "wait",
    "sell",
    "wait",
    "wait",
]
df1

,date,ticker,open,high,low,close,volume,entry_signal,exit_signal
0,2025-04-01,AAPL,219.81,223.68,218.90,223.19,36412700,wait,sell
1,2025-04-02,AAPL,221.32,225.19,221.02,223.89,35905900,buy,wait
2,2025-04-03,AAPL,205.54,207.49,201.25,203.19,103419000,buy,wait
3,2025-04-04,AAPL,193.89,199.88,187.34,188.38,125910900,wait,wait
4,2025-04-07,AAPL,177.20,194.15,174.62,181.46,160466300,wait,sell
5,2025-04-08,AAPL,186.70,190.34,169.21,172.42,120859500,buy,wait
6,2025-04-09,AAPL,171.95,200.61,171.89,198.85,184395900,wait,wait
7,2025-04-10,AAPL,189.07,194.78,183.00,190.42,121880000,wait,sell
8,2025-04-11,AAPL,186.10,199.54,186.06,198.15,87435900,wait,wait
9,2025-04-14,AAPL,211.44,212.94,201.16,202.52,101352900,buy,wait


In [31]:
df1.loc[df1["entry_signal"] == df1["exit_signal"], :]

,date,ticker,open,high,low,close,volume,entry_signal,exit_signal
3,2025-04-04,AAPL,193.89,199.88,187.34,188.38,125910900,wait,wait
6,2025-04-09,AAPL,171.95,200.61,171.89,198.85,184395900,wait,wait
8,2025-04-11,AAPL,186.10,199.54,186.06,198.15,87435900,wait,wait


In [ ]:
df1.to_parquet("../tests/data/sample_gen_trades.parquet", index=False)

temp = pd.read_parquet("../tests/data/sample_gen_trades.parquet")
temp

In [14]:
all(isinstance(dt, datetime) for dt in temp["date"])

True

In [34]:
temp = pd.read_parquet("../tests/data/sample_gen_trades.parquet")
temp

,date,ticker,open,high,low,close,volume,entry_signal,exit_signal
0,2025-04-01,AAPL,219.81,223.68,218.90,223.19,36412700,wait,sell
1,2025-04-02,AAPL,221.32,225.19,221.02,223.89,35905900,buy,wait
2,2025-04-03,AAPL,205.54,207.49,201.25,203.19,103419000,buy,wait
3,2025-04-04,AAPL,193.89,199.88,187.34,188.38,125910900,wait,wait
4,2025-04-07,AAPL,177.20,194.15,174.62,181.46,160466300,wait,sell
5,2025-04-08,AAPL,186.70,190.34,169.21,172.42,120859500,buy,wait
6,2025-04-09,AAPL,171.95,200.61,171.89,198.85,184395900,wait,wait
7,2025-04-10,AAPL,189.07,194.78,183.00,190.42,121880000,wait,sell
8,2025-04-11,AAPL,186.10,199.54,186.06,198.15,87435900,wait,wait
9,2025-04-14,AAPL,211.44,212.94,201.16,202.52,101352900,buy,wait


In [39]:
a = temp.at[0, "date"]
a.to_pydatetime()

datetime.datetime(2025, 4, 1, 0, 0)

In [31]:
datetime(2025, 4, 2, tzinfo=None)

datetime.datetime(2025, 4, 2, 0, 0)

In [32]:
from zoneinfo import ZoneInfo

datetime(2025, 4, 2, tzinfo=ZoneInfo("America/New_York"))

datetime.datetime(2025, 4, 2, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='America/New_York'))

In [52]:
from strat_backtest.base.stock_trade import StockTrade
from collections import deque

first_trade = StockTrade(
    ticker="AAPL",
    entry_datetime="2025-04-02",
    entry_action="buy",
    entry_lots=10,
    entry_price=223.89,
)

second_trade = StockTrade(
    ticker="AAPL",
    entry_datetime=datetime(2025, 4, 3, tzinfo=None),
    entry_action="buy",
    entry_lots=10,
    entry_price=203.19,
)

deque([first_trade, second_trade])

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('223.89'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 3, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('203.19'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [ ]:
from strat_backtest.utils.constants import (
    EntryMethod,
    ExitMethod,
    TrailMethod,
    StopMethod,
)

{val.value: f"strat_backtest.exit_method.{val.value}" for val in ExitMethod}

{'FIFOExit': 'strat_backtest.exit_method.FIFOExit',
 'LIFOExit': 'strat_backtest.exit_method.LIFOExit',
 'HalfFIFOExit': 'strat_backtest.exit_method.HalfFIFOExit',
 'HalfLIFOExit': 'strat_backtest.exit_method.HalfLIFOExit',
 'TakeAllExit': 'strat_backtest.exit_method.TakeAllExit'}

In [67]:
a = "half_fifo_exit"
"".join(val.upper() if val in {"fifo", "lifo"} else val.title() for val in a.split("_"))

'HalfFIFOExit'

In [74]:
from pathlib import Path

a = Path("~/Documents/strat_backtest/src/strat_backtest/exit_method/half_fifo_exit.py")
a

PosixPath('~/Documents/strat_backtest/src/strat_backtest/exit_method/half_fifo_exit.py')

In [81]:
main_pkg = "strat_backtest"

a.parts[::-1].index(main_pkg)

2

In [87]:
b = []
for part in a.parts[::-1]:
    b.append(part)

    if part == main_pkg:
        break

b[::-1]

['strat_backtest', 'exit_method', 'half_fifo_exit.py']

In [4]:
from pathlib import Path

# for rel_path in Path("../src/strat_backtest").iterdir():
#     if rel_path.is_dir():

dir_paths = [
    rel_path
    for rel_path in Path("../src/strat_backtest").iterdir()
    if rel_path.is_dir() and rel_path.name not in {"base", "utils", "__pycache__"}
]
dir_paths

[PosixPath('../src/strat_backtest/exit_method'),
 PosixPath('../src/strat_backtest/trail_method'),
 PosixPath('../src/strat_backtest/entry_method'),
 PosixPath('../src/strat_backtest/stop_method')]

In [12]:
mapping = {}
main_pkg = "strat_backtest"

for folder in dir_paths:
    for file_path in folder.glob("*.py"):
        file_name = file_path.stem
        folder_name = folder.stem

        class_name = "".join(
            part.upper() if part in {"fifo", "lifo"} else part.title()
            for part in file_name.split("_")
        )

        mapping[class_name] = {
            "class_name": class_name,
            "module_path": f"{main_pkg}.{folder_name}.{file_name}",
        }

mapping

{'LIFOExit': {'class_name': 'LIFOExit',
  'module_path': 'strat_backtest.exit_method.lifo_exit'},
 'FIFOExit': {'class_name': 'FIFOExit',
  'module_path': 'strat_backtest.exit_method.fifo_exit'},
 'TakeAllExit': {'class_name': 'TakeAllExit',
  'module_path': 'strat_backtest.exit_method.take_all_exit'},
 'HalfFIFOExit': {'class_name': 'HalfFIFOExit',
  'module_path': 'strat_backtest.exit_method.half_fifo_exit'},
 'HalfLIFOExit': {'class_name': 'HalfLIFOExit',
  'module_path': 'strat_backtest.exit_method.half_lifo_exit'},
 'Init': {'class_name': 'Init',
  'module_path': 'strat_backtest.stop_method.__init__'},
 'FirstTrail': {'class_name': 'FirstTrail',
  'module_path': 'strat_backtest.trail_method.first_trail'},
 'SingleEntry': {'class_name': 'SingleEntry',
  'module_path': 'strat_backtest.entry_method.single_entry'},
 'MultiEntry': {'class_name': 'MultiEntry',
  'module_path': 'strat_backtest.entry_method.multi_entry'},
 'MultiHalfEntry': {'class_name': 'MultiHalfEntry',
  'module_path'